# **Deterministic Policy Gradient for RL-Based MaxCut on Sparse Ising Graphs**

### Habiba Abdelrehim
### Sarah Ameur
### Fadi Younes

README:

First, it is necessary you run cell "1. Libraries" to import all necessary libraries.

Then, it is necessary you run cell "2. Downloading the data for table 1 & 2" to import the datasets for all the graph to be tested in various experiments.

After that, you can choose what cell to run depending on what experiment you want to run.

If you want to test our implementation of the Deterministic-Reinforce algorithm, it is necessary you run cells under "3. Deterministic Reinforce". But first, modify the line "file_name = ..." to input on which graph you want to test the algorithm.

If you want to test our implementation of the Metropolis algorithm, it is necessary you run cells under "4. Benchmark against Metropolis".

If you want to test our implementation of the pure greedy local search Algorithm, it is necessary you run cells under "5. Benchmark against Pure greedy local search".

#**1. Libraries**

In [ ]:
import os
from urllib import request
import scipy.sparse
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import gc
import math
from itertools import product
import requests
import zipfile
import io
from google.colab import runtime
import networkx as nx
import time
import scipy.sparse as sp

device = "cuda" if torch.cuda.is_available() else "cpu"  # choose GPU if present, else CPU
print("Running on:", device)

#**2. Downloading the data for table 1 & 2**


In [ ]:
# Code to download all the graphs datasets to do the experiments, you just have to run this cell to download all the dataset, no need to get them from the websites

dataFile_id = "1d8i-Ylt1TjoejHz3rVw8Omm7OgJEohFA"
dest_folder='./'
URL = "https://docs.google.com/uc?export=download"
session = requests.Session()
response = session.get(URL, params={'id': dataFile_id}, stream=True)


with zipfile.ZipFile(io.BytesIO(response.content)) as z:
    os.makedirs(dest_folder, exist_ok=True)
    z.extractall(dest_folder)

print(f"All data files extracted")

##**Optsicom graphs (Table 1)**

* The Optsicom graphs are sparse random-generated graphs developped as part of the Optsicom project. The data designed for MaxCut problems can be downloaded from: https://grafo.etsii.urjc.es/optsicom/maxcut.html, under "Set2".
* 1. Go to Optsicom website. > 2. Left tab: Optimization Problems. > 3. Under "Set2": Download the data. > 4. Upload it on your local environment in Colab. Make sure to give files the same names, example: "g54100.mc". The correct files are the first 10 of the list.

##**Gset graphs (Table 2)**

* The *Gset* dataset contains graphs with sparse random structure. It can be found here: https://web.stanford.edu/~yyye/yyye/Gset/.
* To understand structure of the Gset data, check out: https://medium.com/toshiba-sbm/benchmarking-the-max-cut-problem-on-the-simulated-bifurcation-machine-e26e1127c0b0.
  * The data is space-separated.
  * The first row of every file contains the number of vertices followed by the number of edges.
  * Each row after that contains a node $i$, followed by a node $j$, and finally the weight $w_{ij}$ of the edge connecting nodes $i$ and $j$.
* The graphs G14, G15, G22, G49, G50, G55, and G70 have edges with uniform weights $w=1$.

#**3. Deterministic Reinforce**

In this section, we attempt to reproduce the results from the paper "Reinforcement Learning for Ising Model", by Lu and Liu https://ml4physicalsciences.github.io/2023/files/NeurIPS_ML4PS_2023_248.pdf

* The input and output layers have $n$ neurons to represent each spin/node of the graph.
* The current spin state $s_t=(\sigma_{1, t}, \sigma_{2, t}, ..., \sigma_{n^2, t})$ is fed to the input layer. After one forward pass, the next state $s_{t+1}$ is produced at the output layer in the form of probabilities to flip each spin: $p_{\theta}(\sigma_i | s_t)$.
* In alignment with deterministic policy update, the next state $s_{t+1}$ is selected based on a fixed threshold of probability 0.5.

$∇_{\theta}J(\theta) = \frac{1}{N}\Sigma_{i=1}^N \Sigma_{t=0}^{T}
  R_t^i∇_\theta \log \pi_\theta(s_{t+1}|s_t)$
   **(9)**

Helper functions help break down Eq. 9 of paper into manageable chunks:
* The cumulative returns $R_t^i$ are calculated using the `hamilton` helper function.
* The log-probability of transition $\log \pi_{\theta}(s_{t+1} | s_t)$ is calculated using the `log_prob` helper function.

* Nodes connected by an edge are spin-spin neighbors.
  * Nodes have spin values $\sigma_i\in\{-1,+1\}$.
  * Coupling is uniform, i.e., $J_{ij} = 1$. This means the weights of every edge is $w_{ij}=1$.
  * Connected nodes $(i,j)\in E$ contribute to the Hamiltonian depending on their relative spins:
    * If $\sigma_i \sigma_j > 0$, the pair increases the Hamiltonian.
    * If $\sigma_i \sigma_j < 0$, the pair lowers the Hamiltonian.
  * Nodes that aren't connected don't interact, therefore don't contribute to the Hamiltonian.

##**Implementation**

In [ ]:
#Choose which graph to use by commenting and uncommenting and changing the string for file_name

# For table 1
file_name = './g54100.mc' # or g54200, g54300...

# For table 2
#file_name = './G14.mc' # or G15, G22...

# Deterministic‑REINFORCE algorithm

# Input hyperparametes to test in their respective list
tries_values        = [1]          # LS moves attempted
T_values            = [10.0]         # Metropolis temperature
steps_values        = [30]                  # Metropolis burn‑in steps
epochs_values       = [25]               # training epochs
batch_values        = [4]                 # parallel chains
traj_values         = [5]                  # trajectory length
lr_values           = [9e-6]            # learning rate
flips_ratio_values  = [0.01]            # LS flips as a fraction of n

grid = product(tries_values,
               T_values,
               steps_values,
               epochs_values,
               batch_values,
               traj_values,
               lr_values,
               flips_ratio_values)

for (tries, T, steps, epochs, batch, traj, lr, flips_ratio) in grid:
    # Unpack the tuple into the value of variables
    tries_value        = tries
    T_value            = T
    steps_value        = steps
    epochs_value       = epochs
    batch_value        = batch
    traj_value         = traj
    lr_value           = lr
    flips_ratio_value  = flips_ratio

    # Convert dense NumPy adjacency to sparse PyTorch tensor on GPU
    def to_sparse(adj):
        sparse_mtx = scipy.sparse.coo_matrix(adj)     # build SciPy sparse matrix
        idx = torch.tensor([sparse_mtx.row, sparse_mtx.col], device=device)   # edge indices as 2*|E| tensor
        val = torch.tensor(sparse_mtx.data, dtype=torch.float32, device=device)  # edge weights
        return torch.sparse_coo_tensor(idx, val, sparse_mtx.shape).coalesce()    # create & coalesce sparse tensor


    # Read .mc graph into dense NumPy adjacency
    def parse_mc(fname):
        with open(fname) as f:
            n, _ = map(int, f.readline().split())   # first line: nbr of nodes, dummy value
            A = np.zeros((n, n), dtype=np.float32)    # allocate n*n zeros matrix
            for line in f:        # remaining lines: u v w
                u, v, w = map(int, line.split())
                A[u - 1, v - 1] = w      # 1‑indexed file so 0‑indexed Python
                A[v - 1, u - 1] = w      # undirected graph then symmetric
        return A

    # Compute Ising Hamiltonian H(s)= 1/4 sum of A_ij sigma_i * sigma_j for a batch of spin vectors
    def hamilton(spins, A):
        if spins.dim() == 1:
            spins = spins.unsqueeze(0)     # promote 1D vector → batch size 1
        A = A.coalesce()    # ensure indices / values are compact
        idx, val = A.indices(), A.values()   # edge list & weights
        prod = spins[:, idx[0]] * spins[:, idx[1]] * val   # element‑wise sigma_i sigma_j * w_ij
        return 0.5 * prod.sum(dim=1)  # returns 1D tensor of size B


    # Compute Max‑Cut value = 1/4 sum of A_ij(1−sigma_i*sigma_j) for a batch
    @torch.no_grad()
    def cutsize(spins, A):
        if spins.dim() == 1:
            spins = spins.unsqueeze(0)
        A = A.coalesce()
        idx, val = A.indices(), A.values()
        prod = (1.0 - spins[:, idx[0]] * spins[:, idx[1]]) * val    # (1−sigma_i*sigma_j) * w_ij
        return 0.25 * prod.sum(dim=1)


    # Batch greedy flip local search: random‑flip bits; accept whole move if H decreases. Repeat 'tries' number of times.
    @torch.no_grad()
    def local_search(spins, A, *, flips=100, tries=tries_value):
        B, n = spins.shape       # B=batch size, n=#nodes
        best = spins.clone()      # best known states so far
        best_H = hamilton(best, A)  # their energies
        A = A.coalesce()
        idxE, valE = A.indices(), A.values()   # local vars for speed
        for _ in range(tries):
            trial = best.clone()   # start from current best
            flip = torch.randint(0, n, (B, flips), device=device)   # random indices per batch row
            rows = torch.arange(B, device=device).unsqueeze(1).expand(-1, flips).reshape(-1)  # row ids
            trial[rows, flip.reshape(-1)] *= -1  # flip chosen spins
            Ht = hamilton(trial, A)   # energy after flip
            mask = Ht < best_H    # improvement mask
            best[mask] = trial[mask]  # accept better configs
            best_H[mask] = Ht[mask]   # update best energies
        return best       # return locally improved spins

    # Metropolis–Hastings burn‑in: single‑spin updates per batch row. Accept downhill moves or uphill with Boltzmann probability.
    @torch.no_grad()
    def metropolis(spins, A, *, T=T_value, steps=steps_value):
        B, n = spins.shape
        A = A.coalesce()
        rows, cols = A.indices() # source/target node indices per edge
        vals = A.values()
        for _ in range(steps):
            k = torch.randint(0, n, (B,), device=device)    # pick a spin index per batch element
            old = spins[torch.arange(B, device=device), k]      # current spin value (+1/-1)
            dE = torch.zeros(B, device=device)      # delta_E for each batch element
            for b in range(B):     # loop over batch rows (small B)
                mask = rows == k[b]      # edges incident to spin k[b]
                dE[b] = -2.0 * old[b] * (spins[b, cols[mask]] * vals[mask]).sum()  # delta_E formula
            acc = (dE <= 0) | (torch.rand(B, device=device) < torch.exp(-dE / T))  # accept moves ?
            spins[acc, k[acc]] *= -1    # flip accepted spins
        return spins

    # Simple 3‑layer MLP; outputs P(sigma=-1) for each node.
    class PolicyNet(nn.Module):
        def __init__(self, n):
            super().__init__()
            self.net = nn.Sequential(
                nn.Linear(n, n), nn.ReLU(),  # hidden layer 1
                nn.Linear(n, n), nn.ReLU(),  # hidden layer 2
                nn.Linear(n, n), nn.Sigmoid()   # outputs in (0,1)
            )
        def forward(self, x):
            return self.net(x)    # forward pass: B*N then B*N probabilities

    # Log‑likelihood of deterministic choice (threshold 0.5) given probabilities p
    def log_prob(p, spins):
        eps = 1e-9
        return torch.where(spins == -1, (p + eps).log(), (1 - p + eps).log()).sum(dim=1)


    # Main deterministic‑REINFORCE training loop. A: sparse adjacency; returns (history of H, best cut found).
    def train(A, epochs=epochs_value, batch=batch_value, traj=traj_value, lr=lr_value, flips_ratio=flips_ratio_value):
        n = A.shape[0]       # number of nodes
        net = PolicyNet(n).to(device)    # policy network
        opt = optim.Adam(net.parameters(), lr=lr)     # Adam optimiser
        sched = optim.lr_scheduler.CosineAnnealingLR(opt, epochs) # cosine LR schedule

        best_cut, hist = -math.inf, []   # track best cut & H history
        for ep in range(1, epochs + 1):  # training epochs
            s = 2 * torch.randint(0, 2, (batch, n), device=device) - 1  # random +/-1 spins
            s = metropolis(s, A)  # burn‑in via Metropolis
            log_pi, rewards = [], []  # per‑step logs & rewards

            for _ in range(traj):     # trajectory of length T
                p = net(s.float())   # forward pass → probabilities
                nxt = torch.where(p > 0.5, -torch.ones_like(p), torch.ones_like(p)).detach()  # deterministic action
                s_hat = local_search(s, A, flips=int(flips_ratio * n), tries=tries_value)  # LS on current
                nxt_hat = local_search(nxt, A, flips=int(flips_ratio * n), tries=tries_value)  # LS on next
                r = hamilton(s_hat, A) - hamilton(nxt_hat, A)     # reward = energy decrease
                log_pi.append(log_prob(p, nxt))  # store log‑prob
                rewards.append(r)  # store reward
                s = nxt_hat     # advance to next state

            R = torch.flip(torch.cumsum(torch.flip(torch.stack(rewards), [0]), 0), [0])  # discounted‑sum
            base = R.mean(dim=1, keepdim=True)    # baseline for variance reduction
            adv = (R - base).detach()  # advantage (no grad)
            loss = -(adv * torch.stack(log_pi)).sum() / batch    # REINFORCE loss (minimise −J)
            opt.zero_grad(set_to_none=True)   # clear gradients
            loss.backward()  # backprop
            opt.step()  # optimiser step
            sched.step()   # LR scheduler step

            cur_H = hamilton(s, A).mean().item()  # mean energy of final batch state
            hist.append(cur_H)    # add to history

            # evaluation every 25 epochs
            if ep % 25 == 0 or ep == epochs:
                with torch.no_grad():
                    test = 2 * torch.randint(0, 2, (batch, n), device=device) - 1
                    test = metropolis(test, A) # burn‑in
                    p = net(test.float())
                    nxt = torch.where(p > 0.5, -torch.ones_like(p), torch.ones_like(p))
                    nxt = local_search(nxt, A, flips=int(flips_ratio * n), tries=500)  # strong LS
                    val = cutsize(nxt, A).max().item()  # best cut in batch
                    best_cut = max(best_cut, val)      # update global best
                print(f"[{ep:4d}/{epochs}]  ⟨H⟩={cur_H:7.2f}   best Cut={best_cut:7.1f}")

            gc.collect()   # release unused tensors
            torch.cuda.empty_cache()     # clear cached GPU memory
        return hist, best_cut     # return training curve & best value


    # Run training
    A_np = parse_mc(file_name)    # load adjacency as dense NumPy array
    A = to_sparse(A_np)    # convert to sparse PyTorch tensor

    print(f"Graph {file_name}: n = {A.shape[0]}, |E| = {A.indices().shape[1]}")  # report size

    hist, best_cut = train(A)      # start training with default params

    print(f"\nFinal best Cutsize on {file_name}: {best_cut:.1f}")  # final result

    # Titles
    title_str = (
        f"{file_name}  |  "
        f"bestCut={best_cut:.1f}  |  "
        f"tries={tries_value}, "
        f"T={T_value}, steps={steps_value}, "
        f"epochs={epochs_value}, batch={batch_value}, traj={traj_value}, "
        f"lr={lr_value}, flips_ratio={flips_ratio_value}"
    )

    # Plot mean H vs epoch
    plt.figure(figsize=(10, 5), dpi=100)
    plt.plot(hist)
    plt.xlabel("epoch")
    plt.ylabel("mean H")
    plt.title(title_str)
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.show()


#♻ Garbage collection

In [ ]:
gc.collect()
torch.cuda.empty_cache()

#**4. Benchmark against Metropolis**

**HOW THE METROPOLIS ALGORITHM WORKS:**
* This is the Metropolis algorithm for the antiferromagnetic Ising spin glass model.
* It simulates graph lattice of $n$ nodes connected with edges, where each node has a spin $\pm 1$.
* The spins are initialized randomly.
* The Hamiltonian is computed by summing spin interactions between nearest neighbors.
* The $Cutsize$ function counts how many neighboring spins are opposite, which is the target configuration minimizing the Hamiltonian in antiferromagnetic materials.
* The algorithm selects a site at random and calculates the energy change if we were to flip it. If flipping lowers the energy, the spin is flipped.
If flipping increases energy, the spin is flipped with Boltzmann probability $p = e^{\Delta E / k_B T}$. The higher the temperature $T$, the more the exploration.
* The process is repeated for multiple iterations to reach thermal equilibrium at the given temperature $T$. At low temperatures $T ⟶ 0$, the configuration that opposes connected spins the most is favored.
At high temperatures $T ⟶ ∞$, spins are randomly oriented due to thermal fluctuations. Indeed, $T$ is the standard deviation of the thermal energy distribution.







##**Metropolis implementation**

* The main implementation in Section 3. uses helper functions that are adapted to Torch tensors.
* Here, for benchmarking against Metropolis, we **rewrote some of the helper functions** so that they handle NumPy arrays instead.

In [ ]:
# Read .mc graph into dense NumPy adjacency
def parse_mc(fname):
    with open(fname) as f:
        n, _ = map(int, f.readline().split())   # first line: nbr of nodes, dummy value
        A = np.zeros((n, n), dtype=np.float32)    # allocate n*n zeros matrix
        for line in f:        # remaining lines: u v w
            u, v, w = map(int, line.split())
            A[u - 1, v - 1] = w      # 1‑indexed file so 0‑indexed Python
            A[v - 1, u - 1] = w      # undirected graph then symmetric
    return A

# random initialization of spins
def rnd_spins(n):
    return np.random.choice([-1, 1], n)


# hamiltonian for numpy arrays
def H(s, A):
    s = np.asarray(s)
    return 0.5 * (s @ A.dot(s) if sp.issparse(A) else np.sum(A * np.outer(s, s)))


def cut(s, A):
    s = np.asarray(s)
    if sp.issparse(A):
        return 0.25 * A.multiply(1 - np.outer(s, s)).sum()      # 1/2 * 1/2 to avoid overcount
    return 0.25 * np.sum(A * (1 - np.outer(s, s)))


# Metropolis algo
def metro(s, A, T, steps=1_000):
    s = np.asarray(s)
    n = len(s)
    sparse = sp.issparse(A)
    for _ in range(steps):
        i = np.random.randint(n)
        dE = -2*s[i]*(A.getrow(i).toarray().dot(s).item() if sparse else A[i] @ s)
        if dE <= 0 or np.random.rand() < np.exp(-dE/T):
            s[i] *= -1    # flip spin
    return s


def plot_line(s, ttl):
    plt.figure(figsize=(8,2))
    plt.imshow(s[np.newaxis,:], cmap='coolwarm', vmin=-1, vmax=1, aspect='auto')
    plt.title(ttl); plt.xticks([]); plt.yticks([]); plt.tight_layout(); plt.show()


# visual samples: graphs with colored nodes
def plot_net(A, s, ttl):
    G = nx.from_scipy_sparse_array(sp.csr_matrix(A))
    pos = nx.spring_layout(G, seed=1)
    nx.draw_networkx_nodes(G, pos, node_color=['red' if x==1 else 'blue' for x in s], node_size=50)
    nx.draw_networkx_edges(G, pos, alpha=.3)
    plt.title(ttl); plt.axis('off'); plt.show()


def run_metro(A, T=0.0, sweeps=500, name="", mode="line", Ts=None):
    n = A.shape[0]
    if mode in ("heat","heatmap"):
        Ts = Ts or [T]
        fig, ax = plt.subplots(1, len(Ts), figsize=(4*len(Ts),2))
        ax = [ax] if len(Ts)==1 else ax
        for a, t in zip(ax, Ts):
            s = rnd_spins(n)    # initialize graph of spins
            s = metro(s, A, t, sweeps*n)    # run Metropolis
            a.imshow(s[np.newaxis,:], cmap='coolwarm', vmin=-1, vmax=1, aspect='auto')
            a.set_title(f"{name}  T={t}  H={H(s,A):.2f}  Cut={cut(s,A):.2f}")
            a.set_yticks([]); a.set_xlabel('idx')
        plt.tight_layout(); plt.show()
        return

    # ONE temperature
    s = rnd_spins(n)
    s = metro(s, A, T, sweeps*n)
    ttl = f"{name}  T={T}  H={H(s,A):.2f}  Cut={cut(s,A):.2f}"

    if mode=="graph":
        plot_net(A, s, ttl)
    else:
        plot_line(s, ttl)

    return s, H(s, A), cut(s, A), T



##**Results Optsicom**

In [ ]:
# optsicom graphs
ids = ["54100","54200","54300","54400","54500",
       "54600","54700","54800","54900","541000"]

for gid in ids:
    adj = parse_mc(f"g{gid}.mc")
    run_metro(adj, T=0.0,          # ground state config
                   sweeps=500, name=f"G{gid}",
                   mode="heatmap", Ts=[0.0])

# some graphs for the video presentation
for gid in ids[:2]:
    adj = parse_mc(f"g{gid}.mc")
    run_metro(adj, T=0.0, sweeps=500, name=f"G{gid}", mode="line")
    run_metro(adj, T=0.0, sweeps=500, name=f"G{gid}", mode="graph")


##**Results Gset**

In [ ]:
ids = [14, 15, 22, 49, 50, 55, 70]

for gid in ids:
    A = parse_mc(f"G{gid}.mc")
    run_metro(A, T=0.0, sweeps=600, name=f"G{gid}", mode="line")


#**5. Benchmark against Pure greedy local search**

##**Results Optsicom**

In [ ]:
#Table 1
ids = [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]

# Read .mc graph into dense NumPy adjacency
def parse_mc(fname):
    with open(fname) as f:
        n, _ = map(int, f.readline().split())   # first line: nbr of nodes, dummy value
        A = np.zeros((n, n), dtype=np.float32)    # allocate n*n zeros matrix
        for line in f:        # remaining lines: u v w
            u, v, w = map(int, line.split())
            A[u - 1, v - 1] = w      # 1‑indexed file so 0‑indexed Python
            A[v - 1, u - 1] = w      # undirected graph then symmetric
    return A

# Convert dense NumPy adjacency to sparse PyTorch tensor on GPU
def to_sparse(adj):
    sparse_mtx = scipy.sparse.coo_matrix(adj)     # build SciPy sparse matrix
    idx = torch.tensor([sparse_mtx.row, sparse_mtx.col], device=device)   # edge indices as 2*|E| tensor
    val = torch.tensor(sparse_mtx.data, dtype=torch.float32, device=device)  # edge weights
    return torch.sparse_coo_tensor(idx, val, sparse_mtx.shape).coalesce()    # create & coalesce sparse tensor

# Compute Max‑Cut value = 1/4 sum of A_ij(1−sigma_i*sigma_j) for a batch
@torch.no_grad()
def cutsize(spins, A):
    if spins.dim() == 1:
        spins = spins.unsqueeze(0)
    A = A.coalesce()
    idx, val = A.indices(), A.values()
    prod = (1.0 - spins[:, idx[0]] * spins[:, idx[1]]) * val    # (1−sigma_i*sigma_j) * w_ij
    return 0.25 * prod.sum(dim=1)

def greedy(ids):
    for g in ids:
        # dense np array --> sparse tensor
        A_np = parse_mc(f"g54{g}.mc")
        A    = to_sparse(A_np)
        n    = A.shape[0]
        # count undirected edges
        E = A.coalesce().indices().shape[1] // 2

        print(f"\ng54{g}: n={n}, |E|={E}")

        # start all +1 spins
        state     = torch.ones((1, n), device=device)
        best_cut  = cutsize(state, A).item()

        for t in range(E):
            improved = False
            # try flipping each spin
            for i in range(n):
                cand = state.clone()
                cand[0, i] *= -1
                c = cutsize(cand, A).item()
                if c > best_cut:
                    best_cut, state = c, cand
                    improved = True
            if not improved:
                print(f"  stopped after {t} iters")
                break

        print(f"  greedy g54{g} bestCut={best_cut:.1f}")

if __name__ == '__main__':
    greedy(ids)


##**Results Gset**

In [ ]:
# Table 2
ids = [14, 15, 22, 49, 50, 55, 70]

# Read .mc graph into dense NumPy adjacency
def parse_mc(fname):
    with open(fname) as f:
        n, _ = map(int, f.readline().split())   # first line: nbr of nodes, dummy value
        A = np.zeros((n, n), dtype=np.float32)    # allocate n*n zeros matrix
        for line in f:        # remaining lines: u v w
            u, v, w = map(int, line.split())
            A[u - 1, v - 1] = w      # 1‑indexed file so 0‑indexed Python
            A[v - 1, u - 1] = w      # undirected graph then symmetric
    return A

# Convert dense NumPy adjacency to sparse PyTorch tensor on GPU
def to_sparse(adj):
    sparse_mtx = scipy.sparse.coo_matrix(adj)     # build SciPy sparse matrix
    idx = torch.tensor([sparse_mtx.row, sparse_mtx.col], device=device)   # edge indices as 2*|E| tensor
    val = torch.tensor(sparse_mtx.data, dtype=torch.float32, device=device)  # edge weights
    return torch.sparse_coo_tensor(idx, val, sparse_mtx.shape).coalesce()    # create & coalesce sparse tensor

# Compute Max‑Cut value = 1/4 sum of A_ij(1−sigma_i*sigma_j) for a batch
@torch.no_grad()
def cutsize(spins, A):
    if spins.dim() == 1:
        spins = spins.unsqueeze(0)
    A = A.coalesce()
    idx, val = A.indices(), A.values()
    prod = (1.0 - spins[:, idx[0]] * spins[:, idx[1]]) * val    # (1−sigma_i*sigma_j) * w_ij
    return 0.25 * prod.sum(dim=1)

def greedy(ids):
    for g in ids:
        # dense np array --> sparse tensor
        A_np = parse_mc(f"G{g}.mc")
        A    = to_sparse(A_np)
        n    = A.shape[0]
        # count undirected edges
        E = A.coalesce().indices().shape[1] // 2

        print(f"\nG{g}: n={n}, |E|={E}")

        # start all +1 spins
        state     = torch.ones((1, n), device=device)
        best_cut  = cutsize(state, A).item()

        for t in range(E):
            improved = False
            # try flipping each spin
            for i in range(n):
                cand = state.clone()
                cand[0, i] *= -1
                c = cutsize(cand, A).item()
                if c > best_cut:
                    best_cut, state = c, cand
                    improved = True
            if not improved:
                print(f"  stopped after {t} iters")
                break

        print(f"  greedy G{g} bestCut={best_cut:.1f}")

if __name__ == '__main__':
    greedy(ids)
